In [2]:
!pip install selenium

                                              0.0/9.5 MB ? eta -:--:--
                                              0.2/9.5 MB 3.5 MB/s eta 0:00:03
     -                                        0.4/9.5 MB 3.7 MB/s eta 0:00:03
     --                                       0.6/9.5 MB 3.8 MB/s eta 0:00:03
     ---                                      0.7/9.5 MB 3.8 MB/s eta 0:00:03
     ----                                     1.0/9.5 MB 4.1 MB/s eta 0:00:03
     -----                                    1.2/9.5 MB 4.2 MB/s eta 0:00:02
     ------                                   1.5/9.5 MB 4.6 MB/s eta 0:00:02
     -------                                  1.9/9.5 MB 4.9 MB/s eta 0:00:02
     --------                                 2.1/9.5 MB 4.9 MB/s eta 0:00:02
     ---------                                2.3/9.5 MB 4.8 MB/s eta 0:00:02
     ----------                               2.5/9.5 MB 4.8 MB/s eta 0:00:02
     -----------                              2.7/9.5 MB 4.7 MB/s eta 0


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
import warnings
warnings.filterwarnings("ignore")
import time
import pandas as pd
import numpy as np

In [7]:
df1 = pd.DataFrame(columns=['Sno','BHK','Property Type', 'beds', 'bathroom', 'Balcony', 'Address','City','State code','Country' ,'Landmark'])

df2 = pd.DataFrame(columns=['Sno', 'carpetarea','status' ,'floor' ,'transaction type', 'Year Built', 'price per square feet'])

df3 = pd.DataFrame(columns=['Sno' , 'furnished status', 'facing' ,'Car Parking', 'Type of ownership' ,'booking amount' ,'buy total price'])

In [8]:
class getWebDriver:
    def __init__(self):
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_experimental_option("detach", True)
        self.driver = webdriver.Chrome(options=chrome_options)
        
    def makeDriver(self,url):
        self.driver.get(url)
        return self.driver

In [35]:
class open_website:
    def OpenSite(self):
        driver_object = getWebDriver()
        driver = driver_object.makeDriver("https://www.christiesrealestate.com/")
        
        time.sleep(3)
        
        element_to_hover = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[4]/nav/ul/li[1]/a/span')
        
        actions = ActionChains(driver)
        actions.move_to_element(element_to_hover).perform()
        time.sleep(2)
        
        click_sales = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[4]/nav/ul/li[1]/ul/li[1]/a/span')
        click_sales.click()
        return driver

## After exploring and spending more than 3-4 hrs on more than 15 different websites which were provided to us as well as so other from my own knowledge
## I finalized Christie's Real Estate website for scrapping my all data. The reason for my selection was : 
## 1. Although Christie didnt had all the features which we have to get regarding a single property like facing, furnished_status and so on, but it had all the necessary features. 
## 2. Many websites had bot behaviour detection, some websites data is not properly structured and some were overly complexed.

## Below is the list of results I planned to scrape from each of the city/state I found on Christie summing to total : 3408

 # Christie’s Real Estate

### Porto : 300 results
### Bahamas : 156 results
### Montana : 333 results
### Colorado : 420 results
### Illinois : 420 results
### Fairfield county : 409 results
### Chicago : 240 results
### Algarve : 288 results
### Ontario : 420 results
### Lisboa : 420 results

# *************************************************************
# *************************************************************

###  For Porto Location *****  300 done 

In [10]:
# Opening website through a custom class using Seleniu,
driver_obj = open_website()
driver = driver_obj.OpenSite()

search_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[1]/div/div[1]/input')
search_bar.send_keys('Porto')
time.sleep(2)

select_Porto = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[2]/div/div/dl[2]/dd')
select_Porto.click()

time.sleep(3)

click_galary = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[2]/div[3]/div[1]/div/div/a[1]/i')
click_galary.click()

In [11]:
df1_container = []
df2_container = []
df3_container = []
i = 1
for page in range(0,25):
    time.sleep(3)
    print(f"page {page+1}")
    get_all_items = driver.find_elements(By.XPATH, '//div[@class="listing-item__image  listing-item__tabs-container  js-c-thumbnail-carousel"]/a[1]')
    
    for item in get_all_items:
        print(f"item no. {i}")
        try:
            item.send_keys(Keys.CONTROL + Keys.RETURN)
            driver.switch_to.window(driver.window_handles[1])
        except StaleElementReferenceException:
            print("Refreshing page because of StaleElementReferenceException")
            driver.refresh()
            time.sleep(1.5)
            continue

        bhk_type = property_type = bedroom = bathroom = balcony = full_address = postal_code = landmark = np.nan
        interior_area= status = floor = transaction = year_build = price_sqft = np.nan
        furnished_status = facing = garage = ownership_type = booking_amt = price= np.nan
        
        time.sleep(1.5)
        all_list = driver.find_elements(By.XPATH,'//dl[@class="listing-info__box-content"]')
        time.sleep(1.5)
        for tags in range(0,len(all_list)):
            try:
                tag_value = all_list[tags].find_element(By.XPATH, './dt').text.strip().upper()
                if tag_value == 'INTERIOR':
                    interior_area = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'BEDROOMS':
                    bedroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'FULL BATHS':
                    bathroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'PROPERTY TYPE':
                    property_type = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'YEAR BUILT':
                    year_build = all_list[tags].find_element(By.XPATH, './dd').text
                else:
                    pass
            except:
                pass

        try:
            address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[1]').text
        except:
            address = ''
        try:
            region_address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[1]').text
        except:
            region_address = ''
        try:
            postal_code = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[2]/div/div/h1/div[2]/div/span[6]').text
        except:
            postal_code = np.nan
            
        try:
            furnished_status = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            furnished_status = np.nan
        try:
            facing = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[11]/div/h1/div[2]/div/span[4]').text
        except:
            facing = np.nan

        full_address = str(address)
        
        price_items = driver.find_elements(By.XPATH, '//span[@class="price__value  u-ignore-phonenumber"]')
        price_items_len = len(price_items)

        if price_items_len == 1 or price_items_len == 2:
            price = price_items[0].text
        elif price_items_len == 4:
            price = np.nan
        elif price_items_len ==5 or price_items_len ==6:
            price = price_items[0].text
        
        elif price_items_len ==7:
            price = price_items[0].text
            price_sqft = price_items[2].text
        else:
            pass
                
        amanity_list = []
        amenities = driver.find_elements(By.XPATH, '//span[@class="prop-description__amenities-list-item-text"]')

        garage = False
        for amenity in amenities:
            if 'garage' in amenity.text.lower():
                garage = True

        df1_row = [i,bhk_type,property_type,bedroom,bathroom,balcony,full_address,'Porto',postal_code,'Portugal',landmark]
        df2_row = [i,interior_area,status,floor,transaction,year_build,price_sqft]
        df3_row = [i,furnished_status,facing,garage,ownership_type,booking_amt,price]

        df1_container.append(df1_row)
        df2_container.append(df2_row)
        df3_container.append(df3_row)

        print(df1_row)
        print(df2_row)
        print(df3_row)

        i+=1
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    next_page = driver.find_element(By.XPATH, '//a[@class="paging__item  paging__item--next  js-at-icon-paging  js-paging-next"]')
    next_page.click()

page 1
item no. 1
[1, nan, 'Ranch/Farm', nan, nan, nan, 'Farm with views over the River Douro, Sebolido, Portugal', 'Porto', '4575-177', 'Portugal', nan]
[1, nan, nan, nan, nan, nan, nan]
[1, nan, nan, False, nan, nan, nan]
item no. 2
[2, nan, 'Ranch/Farm', nan, nan, nan, "Quinta d'Arauta-Agrotourism with vineyard, for sale, Lousada, Portugal", 'Porto', '4620-100', 'Portugal', nan]
[2, nan, nan, nan, nan, '2024', nan]
[2, nan, nan, False, nan, nan, nan]
item no. 3
[3, nan, nan, nan, nan, nan, 'Property with gardens, swimming pool and river view, for sale, in V. N. Gaia, Portugal. N. Gaia, Portugal', 'Porto', '4430-498', 'Portugal', nan]
[3, nan, nan, nan, nan, nan, nan]
[3, nan, nan, False, nan, nan, nan]
item no. 4
[4, nan, nan, nan, nan, nan, 'Luxury villa for sale, with garden, by the sea, Porto, Portugal', 'Porto', '4150-356', 'Portugal', nan]
[4, nan, nan, nan, nan, nan, nan]
[4, nan, nan, False, nan, nan, nan]
item no. 5
[5, nan, 'Co-op property', nan, nan, nan, 'Sale: building a

In [12]:
print(len(df1_container), len(df2_container), len(df3_container))
for list in df1_container:
    df1.loc[len(df1)] = list

for list in df2_container:
    df2.loc[len(df2)] = list
    
for list in df3_container:
    df3.loc[len(df3)] = list

300 300 300


In [13]:
print(df1.shape, df2.shape, df3.shape)

(300, 11) (300, 7) (300, 7)


### For Bahamas Location ***** 156 done

In [21]:
# Opening website through a custom class using Seleniu,
driver_obj = open_website()
driver = driver_obj.OpenSite()

search_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[1]/div/div[1]/input')
search_bar.send_keys('Bahamas')
time.sleep(2)

select_Bahamas = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[2]/div/div/dl/dd')
select_Bahamas.click()
time.sleep(3)

click_galary = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[2]/div[3]/div[1]/div/div/a[1]/i')
click_galary.click()

In [22]:
df1_container = []
df2_container = []
df3_container = []
i = 1
for page in range(0,13):
    time.sleep(3)
    print(f"page {page+1}")
    get_all_items = driver.find_elements(By.XPATH, '//div[@class="listing-item__image  listing-item__tabs-container  js-c-thumbnail-carousel"]/a[1]')
    
    for item in get_all_items:
        print(f"item no. {i}")
        try:
            item.send_keys(Keys.CONTROL + Keys.RETURN)
            driver.switch_to.window(driver.window_handles[1])
        except StaleElementReferenceException:
            print("Refreshing page because of StaleElementReferenceException")
            driver.refresh()
            time.sleep(1.5)
            continue

        bhk_type = property_type = bedroom = bathroom = balcony = full_address = postal_code = landmark = np.nan
        interior_area= status = floor = transaction = year_build = price_sqft = np.nan
        furnished_status = facing = garage = ownership_type = booking_amt = price= np.nan
        
        time.sleep(1.5)
        all_list = driver.find_elements(By.XPATH,'//dl[@class="listing-info__box-content"]')
        time.sleep(1.5)
        for tags in range(0,len(all_list)):
            try:
                tag_value = all_list[tags].find_element(By.XPATH, './dt').text.strip().upper()
                if tag_value == 'INTERIOR':
                    interior_area = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'BEDROOMS':
                    bedroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'FULL BATHS':
                    bathroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'PROPERTY TYPE':
                    property_type = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'YEAR BUILT':
                    year_build = all_list[tags].find_element(By.XPATH, './dd').text
                else:
                    pass
            except:
                pass
        try:
            address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[1]').text
        except:
            address = ''
        try:
            region_address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[1]').text
        except:
            region_address = ''
        try:
            postal_code = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            postal_code = np.nan
            
        try:
            furnished_status = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            furnished_status = np.nan
        try:
            facing = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[11]/div/h1/div[2]/div/span[4]').text
        except:
            facing = np.nan

        full_address = str(address)
        
        price_items = driver.find_elements(By.XPATH, '//span[@class="price__value  u-ignore-phonenumber"]')
        price_items_len = len(price_items)

        if price_items_len == 1 or price_items_len == 2:
            price = price_items[0].text
        elif price_items_len == 4:
            price = np.nan
        elif price_items_len ==5 or price_items_len ==6:
            price = price_items[0].text
        
        elif price_items_len ==7:
            price = price_items[0].text
            price_sqft = price_items[2].text
        else:
            pass
                
        amanity_list = []
        amenities = driver.find_elements(By.XPATH, '//span[@class="prop-description__amenities-list-item-text"]')

        garage = False
        for amenity in amenities:
            if 'garage' in amenity.text.lower():
                garage = True

        df1_row = [i,bhk_type,property_type,bedroom,bathroom,balcony,full_address,'Bahamas',postal_code,'Bahamas',landmark]
        df2_row = [i,interior_area,status,floor,transaction,year_build,price_sqft]
        df3_row = [i,furnished_status,facing,garage,ownership_type,booking_amt,price]

        df1_container.append(df1_row)
        df2_container.append(df2_row)
        df3_container.append(df3_row)

        print(df1_row)
        print(df2_row)
        print(df3_row)

        i+=1
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    next_page = driver.find_element(By.XPATH, '//a[@class="paging__item  paging__item--next  js-at-icon-paging  js-paging-next"]')
    next_page.click()

page 1
item no. 1
[1, nan, 'Land/Lot', nan, nan, nan, 'Lot 35, Rose Island Beach and Harbour Club - MLS 54546', 'Bahamas', nan, 'Bahamas', nan]
[1, nan, nan, nan, nan, nan, nan]
[1, nan, nan, False, nan, nan, '$1,500,000']
item no. 2
[2, nan, 'Single Family', '7', '7', nan, 'Illawarra House', 'Bahamas', nan, 'Bahamas', nan]
[2, '11,625 Sq Ft.', nan, nan, nan, '2015', '$4,731']
[2, nan, nan, False, nan, nan, '$55,000,000']
item no. 3
[3, nan, 'Land/Lot', nan, nan, nan, 'The Salinas - MLS 57498', 'Bahamas', nan, 'Bahamas', nan]
[3, nan, nan, nan, nan, nan, nan]
[3, nan, nan, False, nan, nan, '$49,500,000']
item no. 4
[4, nan, 'Private Island', '7', '9', nan, 'Royal Island', 'Bahamas', nan, 'Bahamas', nan]
[4, '10,000 Sq Ft.', nan, nan, nan, '2007', '$4,500']
[4, nan, nan, False, nan, nan, '$45,000,000']
item no. 5
[5, nan, 'Private Island', '20', '25', nan, 'Bird Cay, Private Island', 'Bahamas', nan, 'Bahamas', nan]
[5, '50,000 Sq Ft.', nan, nan, nan, nan, nan]
[5, nan, nan, False, nan, 

In [23]:
print(len(df1_container), len(df2_container), len(df3_container))

for list in df1_container:
    df1.loc[len(df1)] = list
    
for list in df2_container:
    df2.loc[len(df2)] = list
    
for list in df3_container:
    df3.loc[len(df3)] = list

print(df1.shape, df2.shape, df3.shape)

156 156 156
(789, 11) (789, 7) (789, 7)


###  For Montana Location ***** 333 done

In [14]:
# Opening website through a custom class using Seleniu,
driver_obj = open_website()
driver = driver_obj.OpenSite()

search_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[1]/div/div[1]/input')
search_bar.send_keys('Montana')
time.sleep(2)

select_Montana = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[2]/div/div/dl[1]/dd')
select_Montana.click()
time.sleep(3)

click_galary = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[2]/div[3]/div[1]/div/div/a[1]/i')
click_galary.click()

In [15]:
df1_container = []
df2_container = []
df3_container = []
i = 1
for page in range(0,35):
    time.sleep(3)
    print(f"page {page+1}")
    get_all_items = driver.find_elements(By.XPATH, '//div[@class="listing-item__image  listing-item__tabs-container  js-c-thumbnail-carousel"]/a[1]')
    
    for item in get_all_items:
        print(f"item no. {i}")
        try:
            item.send_keys(Keys.CONTROL + Keys.RETURN)
            driver.switch_to.window(driver.window_handles[1])
        except StaleElementReferenceException:
            print("Refreshing page because of StaleElementReferenceException")
            driver.refresh()
            time.sleep(1.5)
            continue

        bhk_type = property_type = bedroom = bathroom = balcony = full_address = postal_code = landmark = np.nan
        interior_area= status = floor = transaction = year_build = price_sqft = np.nan
        furnished_status = facing = garage = ownership_type = booking_amt = price= np.nan
        
        time.sleep(1.5)
        all_list = driver.find_elements(By.XPATH,'//dl[@class="listing-info__box-content"]')
        time.sleep(1.5)
        for tags in range(0,len(all_list)):
            try:
                tag_value = all_list[tags].find_element(By.XPATH, './dt').text.strip().upper()
                if tag_value == 'INTERIOR':
                    interior_area = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'BEDROOMS':
                    bedroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'FULL BATHS':
                    bathroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'PROPERTY TYPE':
                    property_type = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'YEAR BUILT':
                    year_build = all_list[tags].find_element(By.XPATH, './dd').text
                else:
                    pass
            except:
                pass
        try:
            address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[1]').text
        except:
            address = ''
        try:
            region_address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[1]').text
        except:
            region_address = ''
        try:
            postal_code = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            postal_code = np.nan
            
        try:
            furnished_status = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            furnished_status = np.nan
        try:
            facing = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[11]/div/h1/div[2]/div/span[4]').text
        except:
            facing = np.nan

        full_address = str(address)
        
        price_items = driver.find_elements(By.XPATH, '//span[@class="price__value  u-ignore-phonenumber"]')
        price_items_len = len(price_items)

        if price_items_len == 1 or price_items_len == 2:
            price = price_items[0].text
        elif price_items_len == 4:
            price = np.nan
        elif price_items_len ==5 or price_items_len ==6:
            price = price_items[0].text
        
        elif price_items_len ==7:
            price = price_items[0].text
            price_sqft = price_items[2].text
        else:
            pass
                
        amanity_list = []
        amenities = driver.find_elements(By.XPATH, '//span[@class="prop-description__amenities-list-item-text"]')

        garage = False
        for amenity in amenities:
            if 'garage' in amenity.text.lower():
                garage = True

        df1_row = [i,bhk_type,property_type,bedroom,bathroom,balcony,full_address,'Montana',postal_code,'USA',landmark]
        df2_row = [i,interior_area,status,floor,transaction,year_build,price_sqft]
        df3_row = [i,furnished_status,facing,garage,ownership_type,booking_amt,price]

        df1_container.append(df1_row)
        df2_container.append(df2_row)
        df3_container.append(df3_row)

        print(df1_row)
        print(df2_row)
        print(df3_row)

        i+=1
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    next_page = driver.find_element(By.XPATH, '//a[@class="paging__item  paging__item--next  js-at-icon-paging  js-paging-next"]')
    next_page.click()

page 1
item no. 1
[1, nan, 'Single Family', '7', '3', nan, '8500 North Fork Road', 'Montana', '59928', 'USA', nan]
[1, '6,520 Sq Ft.', nan, nan, nan, '1980', '$4,563']
[1, nan, nan, True, nan, nan, '$29,750,000']
item no. 2
[2, nan, 'Land/Lot', nan, nan, nan, '514 W 2nd Street', 'Montana', '59937', 'USA', nan]
[2, nan, nan, nan, nan, nan, nan]
[2, nan, nan, False, nan, nan, '$19,988,888']
item no. 3
[3, nan, 'Land/Lot', nan, nan, nan, '424 W 2nd Street', 'Montana', '59937', 'USA', nan]
[3, nan, nan, nan, nan, nan, nan]
[3, nan, nan, False, nan, nan, '$19,988,888']
item no. 4
[4, nan, 'Land/Lot', nan, nan, nan, '438 W 2nd Street', 'Montana', '59937', 'USA', nan]
[4, nan, nan, nan, nan, nan, nan]
[4, nan, nan, False, nan, nan, '$19,988,888']
item no. 5
[5, nan, 'Single Family', '14', '11', nan, '13817 Sylvan Drive', 'Montana', '59911', 'USA', nan]
[5, '13,991 Sq Ft.', nan, nan, nan, '1924', '$1,358']
[5, nan, nan, True, nan, nan, '$19,000,000']
item no. 6
[6, nan, 'Single Family', '5', '

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@class="paging__item  paging__item--next  js-at-icon-paging  js-paging-next"]"}
  (Session info: chrome=126.0.6478.114); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF67C6D3E32+31618]
	(No symbol) [0x00007FF67C64B099]
	(No symbol) [0x00007FF67C50888A]
	(No symbol) [0x00007FF67C558524]
	(No symbol) [0x00007FF67C55862C]
	(No symbol) [0x00007FF67C59F787]
	(No symbol) [0x00007FF67C57D14F]
	(No symbol) [0x00007FF67C59CA80]
	(No symbol) [0x00007FF67C57CEB3]
	(No symbol) [0x00007FF67C54A46B]
	(No symbol) [0x00007FF67C54B001]
	GetHandleVerifier [0x00007FF67C9D9FFD+3202381]
	GetHandleVerifier [0x00007FF67CA26A1D+3516269]
	GetHandleVerifier [0x00007FF67CA1C490+3473888]
	GetHandleVerifier [0x00007FF67C785D36+760454]
	(No symbol) [0x00007FF67C656B3F]
	

In [17]:
print(len(df1_container), len(df2_container), len(df3_container))
for list in df1_container:
    df1.loc[len(df1)] = list

for list in df2_container:
    df2.loc[len(df2)] = list
    
for list in df3_container:
    df3.loc[len(df3)] = list

print(df1.shape)

333 333 333
(633, 11)


###  For Colorado Location ***** 480 done

In [25]:
# Opening website through a custom class using Seleniu,
driver_obj = open_website()
driver = driver_obj.OpenSite()
time.sleep(2)

search_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[1]/div/div[1]/input')
search_bar.send_keys('Colorado')
time.sleep(2)

select_colorado = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[2]/div/div/dl[2]/dd')
select_colorado.click()

time.sleep(3)

click_galary = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[2]/div[3]/div[1]/div/div/a[1]/i')
click_galary.click()

In [26]:
df1_container = []
df2_container = []
df3_container = []
i = 1
for page in range(0,40):
    time.sleep(3)
    print(f"page {page+1}")
    time.sleep(2)
    get_all_items = driver.find_elements(By.XPATH, '//div[@class="listing-item__image  listing-item__tabs-container  js-c-thumbnail-carousel"]/a[1]')
    print(f"total items found on {page+1}")
    for item in get_all_items:
        print(f"item no. {i}")
        try:
            item.send_keys(Keys.CONTROL + Keys.RETURN)
            driver.switch_to.window(driver.window_handles[1])
        except StaleElementReferenceException:
            print("Refreshing page because of StaleElementReferenceException")
            driver.refresh()
            time.sleep(1.5)
            continue

        bhk_type = property_type = bedroom = bathroom = balcony = full_address = postal_code = landmark = np.nan
        interior_area= status = floor = transaction = year_build = price_sqft = np.nan
        furnished_status = facing = garage = ownership_type = booking_amt = price= np.nan
        
        time.sleep(1.5)
        all_list = driver.find_elements(By.XPATH,'//dl[@class="listing-info__box-content"]')
        time.sleep(1.5)
        for tags in range(0,len(all_list)):
            try:
                tag_value = all_list[tags].find_element(By.XPATH, './dt').text.strip().upper()
                if tag_value == 'INTERIOR':
                    interior_area = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'BEDROOMS':
                    bedroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'FULL BATHS':
                    bathroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'PROPERTY TYPE':
                    property_type = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'YEAR BUILT':
                    year_build = all_list[tags].find_element(By.XPATH, './dd').text
                else:
                    pass
            except:
                pass
        try:
            address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[1]').text
        except:
            address = ''
        try:
            region_address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[1]').text
        except:
            region_address = ''
        try:
            postal_code = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            postal_code = np.nan
            
        try:
            furnished_status = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            furnished_status = np.nan
        try:
            facing = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[11]/div/h1/div[2]/div/span[4]').text
        except:
            facing = np.nan

        full_address = str(address)
        
        price_items = driver.find_elements(By.XPATH, '//span[@class="price__value  u-ignore-phonenumber"]')
        price_items_len = len(price_items)

        if price_items_len == 1 or price_items_len == 2:
            price = price_items[0].text
        elif price_items_len == 4:
            price = np.nan
        elif price_items_len ==5 or price_items_len ==6:
            price = price_items[0].text
        
        elif price_items_len ==7:
            price = price_items[0].text
            price_sqft = price_items[2].text
        else:
            pass
                
        amanity_list = []
        amenities = driver.find_elements(By.XPATH, '//span[@class="prop-description__amenities-list-item-text"]')

        garage = False
        for amenity in amenities:
            if 'garage' in amenity.text.lower():
                garage = True

        df1_row = [i,bhk_type,property_type,bedroom,bathroom,balcony,full_address,'Colorado',postal_code,'USA',landmark]
        df2_row = [i,interior_area,status,floor,transaction,year_build,price_sqft]
        df3_row = [i,furnished_status,facing,garage,ownership_type,booking_amt,price]

        df1_container.append(df1_row)
        df2_container.append(df2_row)
        df3_container.append(df3_row)

        print(df1_row)
        print(df2_row)
        print(df3_row)

        i+=1
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    next_page = driver.find_element(By.XPATH, '//a[@class="paging__item  paging__item--next  js-at-icon-paging  js-paging-next"]')
    next_page.click()

page 1
total items found on 1
item no. 1
[1, nan, nan, nan, nan, nan, '810 W Smuggler Street', 'Colorado', '81611', 'USA', nan]
[1, nan, nan, nan, nan, nan, '$4,698']
[1, nan, nan, False, nan, nan, '$28,500,000']
item no. 2
[2, nan, 'Single Family', '5', '5', nan, '804 Hunter Creek Road', 'Colorado', '81611', 'USA', nan]
[2, '4,920 Sq Ft.', nan, nan, nan, '1996', '$5,081']
[2, nan, nan, False, nan, nan, '$25,000,000']
item no. 3
[3, nan, 'Single Family', '4', '6', nan, '349 Draw Drive', 'Colorado', '81611', 'USA', nan]
[3, '4,744 Sq Ft.', nan, nan, nan, '1997', '$4,690']
[3, nan, nan, True, nan, nan, '$22,250,000']
item no. 4
[4, nan, 'Single Family', '5', '8', nan, '30 Maroon Court', 'Colorado', '81611', 'USA', nan]
[4, '5,066 Sq Ft.', nan, nan, nan, '2023', '$3,652']
[4, nan, nan, False, nan, nan, '$18,500,000']
item no. 5
[5, nan, nan, nan, nan, nan, '411 Wood Road, #8', 'Colorado', '81615', 'USA', nan]
[5, nan, nan, nan, nan, nan, '$5,001']
[5, nan, nan, True, nan, nan, '$16,800,00

In [27]:
print(len(df1_container), len(df2_container), len(df3_container))
for list in df1_container:
    df1.loc[len(df1)] = list

for list in df2_container:
    df2.loc[len(df2)] = list
    
for list in df3_container:
    df3.loc[len(df3)] = list

print(df1.shape, df2.shape, df3.shape)

480 480 480
(1269, 11) (1269, 7) (1269, 7)


###  For Illinois Location *****  420 done

In [28]:
# Opening website through a custom class using Seleniu,
driver_obj = open_website()
driver = driver_obj.OpenSite()

search_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[1]/div/div[1]/input')
search_bar.send_keys('Illinois')
time.sleep(2)

select_Illinois = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[2]/div/div/dl[1]/dd')
select_Illinois.click()

time.sleep(3)

click_galary = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[2]/div[3]/div[1]/div/div/a[1]/i')
click_galary.click()

In [29]:
df1_container = []
df2_container = []
df3_container = []
i = 1
for page in range(0,35):
    time.sleep(3)
    print(f"page {page+1}")
    get_all_items = driver.find_elements(By.XPATH, '//div[@class="listing-item__image  listing-item__tabs-container  js-c-thumbnail-carousel"]/a[1]')
    
    for item in get_all_items:
        print(f"item no. {i}")
        try:
            item.send_keys(Keys.CONTROL + Keys.RETURN)
            driver.switch_to.window(driver.window_handles[1])
        except StaleElementReferenceException:
            print("Refreshing page because of StaleElementReferenceException")
            driver.refresh()
            time.sleep(1.5)
            continue

        bhk_type = property_type = bedroom = bathroom = balcony = full_address = postal_code = landmark = np.nan
        interior_area= status = floor = transaction = year_build = price_sqft = np.nan
        furnished_status = facing = garage = ownership_type = booking_amt = price= np.nan
        
        time.sleep(1.5)
        all_list = driver.find_elements(By.XPATH,'//dl[@class="listing-info__box-content"]')
        time.sleep(1.5)
        for tags in range(0,len(all_list)):
            try:
                tag_value = all_list[tags].find_element(By.XPATH, './dt').text.strip().upper()
                if tag_value == 'INTERIOR':
                    interior_area = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'BEDROOMS':
                    bedroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'FULL BATHS':
                    bathroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'PROPERTY TYPE':
                    property_type = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'YEAR BUILT':
                    year_build = all_list[tags].find_element(By.XPATH, './dd').text
                else:
                    pass
            except:
                pass
        try:   
            address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[1]').text
        except:
            address = ''
        try:
            region_address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[1]').text
        except:
            region_address = ''
        try:
            postal_code = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            postal_code = np.nan
            
        try:
            furnished_status = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            furnished_status = np.nan
        try:
            facing = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[11]/div/h1/div[2]/div/span[4]').text
        except:
            facing = np.nan

        full_address = str(address)
        
        price_items = driver.find_elements(By.XPATH, '//span[@class="price__value  u-ignore-phonenumber"]')
        price_items_len = len(price_items)

        if price_items_len == 1 or price_items_len == 2:
            price = price_items[0].text
        elif price_items_len == 4:
            price = np.nan
        elif price_items_len ==5 or price_items_len ==6:
            price = price_items[0].text
        
        elif price_items_len ==7:
            price = price_items[0].text
            price_sqft = price_items[2].text
        else:
            pass
                
        amanity_list = []
        amenities = driver.find_elements(By.XPATH, '//span[@class="prop-description__amenities-list-item-text"]')

        garage = False
        for amenity in amenities:
            if 'garage' in amenity.text.lower():
                garage = True

        df1_row = [i,bhk_type,property_type,bedroom,bathroom,balcony,full_address,'Illinois',postal_code,'USA',landmark]
        df2_row = [i,interior_area,status,floor,transaction,year_build,price_sqft]
        df3_row = [i,furnished_status,facing,garage,ownership_type,booking_amt,price]

        df1_container.append(df1_row)
        df2_container.append(df2_row)
        df3_container.append(df3_row)

        print(df1_row)
        print(df2_row)
        print(df3_row)

        i+=1
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    next_page = driver.find_element(By.XPATH, '//a[@class="paging__item  paging__item--next  js-at-icon-paging  js-paging-next"]')
    next_page.click()

page 1
item no. 1
[1, nan, 'Single Family', '6', '6', nan, '700 Crab Tree Farm Lane', 'Illinois', '60044', 'USA', nan]
[1, '9,990 Sq Ft.', nan, nan, nan, '1994', '$1,992']
[1, nan, nan, False, nan, nan, '$19,900,000']
item no. 2
[2, nan, 'Estate', nan, nan, nan, '1125 Longvalley Estate', 'Illinois', nan, 'USA', nan]
[2, nan, nan, nan, nan, '1927', nan]
[2, nan, nan, True, nan, nan, '$12,750,000']
item no. 3
[3, nan, nan, nan, nan, nan, '139 SHERIDAN Road', 'Illinois', '60093', 'USA', nan]
[3, nan, nan, nan, nan, nan, '$926']
[3, nan, nan, False, nan, nan, '$11,750,000']
item no. 4
[4, nan, 'Single Family', '6', '5', nan, '1867 N Burling Street', 'Illinois', ',', 'USA', nan]
[4, '9,000 Sq Ft.', nan, nan, nan, '2024', '$1,111']
[4, nan, nan, False, nan, nan, '$10,000,000']
item no. 5
[5, nan, 'Single Family', '10', '8', nan, '2026 N Kenmore Avenue', 'Illinois', ',', 'USA', nan]
[5, '13,400 Sq Ft.', nan, nan, nan, '1885', '$746']
[5, nan, nan, False, nan, nan, '$9,995,000']
item no. 6
[6,

In [30]:
print(len(df1_container), len(df2_container), len(df3_container))
for list in df1_container:
    df1.loc[len(df1)] = list

for list in df2_container:
    df2.loc[len(df2)] = list
    
for list in df3_container:
    df3.loc[len(df3)] = list

print(df1.shape)

420 420 420
(1689, 11)


###  For Fairfield county Location *****  409 done

In [37]:
# Opening website through a custom class using Seleniu,
driver_obj = open_website()
driver = driver_obj.OpenSite()

search_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[1]/div/div[1]/input')
search_bar.send_keys('Fairfield county')
time.sleep(2)

select_Fairfield = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[2]/div/div/dl/dd')
select_Fairfield.click()

time.sleep(3)

click_galary = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[2]/div[3]/div[1]/div/div/a[1]/i')
click_galary.click()

In [38]:
df1_container = []
df2_container = []
df3_container = []
i = 1
for page in range(0,35):
    time.sleep(3)
    print(f"page {page+1}")
    get_all_items = driver.find_elements(By.XPATH, '//div[@class="listing-item__image  listing-item__tabs-container  js-c-thumbnail-carousel"]/a[1]')
    
    for item in get_all_items:
        print(f"item no. {i}")
        try:
            item.send_keys(Keys.CONTROL + Keys.RETURN)
            driver.switch_to.window(driver.window_handles[1])
        except StaleElementReferenceException:
            print("Refreshing page because of StaleElementReferenceException")
            driver.refresh()
            time.sleep(1.5)
            continue

        bhk_type = property_type = bedroom = bathroom = balcony = full_address = postal_code = landmark = np.nan
        interior_area= status = floor = transaction = year_build = price_sqft = np.nan
        furnished_status = facing = garage = ownership_type = booking_amt = price= np.nan
        
        time.sleep(1.5)
        all_list = driver.find_elements(By.XPATH,'//dl[@class="listing-info__box-content"]')
        time.sleep(1.5)
        for tags in range(0,len(all_list)):
            try:
                tag_value = all_list[tags].find_element(By.XPATH, './dt').text.strip().upper()
                if tag_value == 'INTERIOR':
                    interior_area = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'BEDROOMS':
                    bedroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'FULL BATHS':
                    bathroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'PROPERTY TYPE':
                    property_type = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'YEAR BUILT':
                    year_build = all_list[tags].find_element(By.XPATH, './dd').text
                else:
                    pass
            except:
                pass
        try:
            address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[1]').text
        except:
            address = ''
        try:
            region_address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[1]').text
        except:
            region_address = ''
        try:
            postal_code = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            postal_code = np.nan
            
        try:
            furnished_status = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            furnished_status = np.nan
        try:
            facing = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[11]/div/h1/div[2]/div/span[4]').text
        except:
            facing = np.nan

        full_address = str(address)
        
        price_items = driver.find_elements(By.XPATH, '//span[@class="price__value  u-ignore-phonenumber"]')
        price_items_len = len(price_items)

        if price_items_len == 1 or price_items_len == 2:
            price = price_items[0].text
        elif price_items_len == 4:
            price = np.nan
        elif price_items_len ==5 or price_items_len ==6:
            price = price_items[0].text
        
        elif price_items_len ==7:
            price = price_items[0].text
            price_sqft = price_items[2].text
        else:
            pass
                
        amanity_list = []
        amenities = driver.find_elements(By.XPATH, '//span[@class="prop-description__amenities-list-item-text"]')

        garage = False
        for amenity in amenities:
            if 'garage' in amenity.text.lower():
                garage = True

        df1_row = [i,bhk_type,property_type,bedroom,bathroom,balcony,full_address,'Fairfield County',postal_code,'USA',landmark]
        df2_row = [i,interior_area,status,floor,transaction,year_build,price_sqft]
        df3_row = [i,furnished_status,facing,garage,ownership_type,booking_amt,price]

        df1_container.append(df1_row)
        df2_container.append(df2_row)
        df3_container.append(df3_row)

        print(df1_row)
        print(df2_row)
        print(df3_row)

        i+=1
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    next_page = driver.find_element(By.XPATH, '//a[@class="paging__item  paging__item--next  js-at-icon-paging  js-paging-next"]')
    next_page.click()

page 1
item no. 1
[1, nan, 'Commercial / Office', nan, nan, nan, '1137-41 East Putnam Avenue & 5 Sheephill Road', 'Fairfield County', '06878', 'USA', nan]
[1, nan, nan, nan, nan, nan, nan]
[1, nan, nan, False, nan, nan, '$7,995,000']
item no. 2
[2, nan, 'Land / Commercial', nan, nan, nan, '1137-41 East Putnam Avenue & 5 Sheephill Road, Riverside, CT 06878', 'Fairfield County', '06878', 'USA', nan]
[2, nan, nan, nan, nan, nan, nan]
[2, nan, nan, False, nan, nan, '$7,995,000']
item no. 3
[3, nan, 'Single Family', '4', '4', nan, '32 Hunting Ridge', 'Fairfield County', ',', 'USA', nan]
[3, '5,074 Sq Ft.', nan, nan, nan, '2002', '$320']
[3, nan, nan, False, nan, nan, '$1,625,000']
item no. 4
[4, nan, 'Multi Family', '7', '3', nan, '32 Arther Street', 'Fairfield County', '06831', 'USA', nan]
[4, '2,836 Sq Ft.', nan, nan, nan, '1965', '$494']
[4, nan, nan, False, nan, nan, '$1,400,000']
item no. 5
[5, nan, 'Single Family', '6', '6', nan, '545 Indian Field', 'Fairfield County', '06830', 'USA',

In [39]:
print(len(df1_container), len(df2_container), len(df3_container))
for list in df1_container:
    df1.loc[len(df1)] = list

for list in df2_container:
    df2.loc[len(df2)] = list
    
for list in df3_container:
    df3.loc[len(df3)] = list

print(df1.shape)

409 409 409
(2098, 11)


###  For Chicago Location *****  240 done

In [40]:
# Opening website through a custom class using Seleniu,
driver_obj = open_website()
driver = driver_obj.OpenSite()

search_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[1]/div/div[1]/input')
search_bar.send_keys('Chicago')
time.sleep(2)

select_Chicago = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[2]/div/div/dl[1]/dd')
select_Chicago.click()

time.sleep(3)

click_galary = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[2]/div[3]/div[1]/div/div/a[1]/i')
click_galary.click()

In [41]:
df1_container = []
df2_container = []
df3_container = []
i = 1
for page in range(0,20):
    time.sleep(3)
    print(f"page {page+1}")
    get_all_items = driver.find_elements(By.XPATH, '//div[@class="listing-item__image  listing-item__tabs-container  js-c-thumbnail-carousel"]/a[1]')
    
    for item in get_all_items:
        print(f"item no. {i}")
        try:
            item.send_keys(Keys.CONTROL + Keys.RETURN)
            driver.switch_to.window(driver.window_handles[1])
        except StaleElementReferenceException:
            print("Refreshing page because of StaleElementReferenceException")
            driver.refresh()
            time.sleep(1.5)
            continue

        bhk_type = property_type = bedroom = bathroom = balcony = full_address = postal_code = landmark = np.nan
        interior_area= status = floor = transaction = year_build = price_sqft = np.nan
        furnished_status = facing = garage = ownership_type = booking_amt = price= np.nan
        
        time.sleep(1.5)
        all_list = driver.find_elements(By.XPATH,'//dl[@class="listing-info__box-content"]')
        time.sleep(1.5)
        for tags in range(0,len(all_list)):
            try:
                tag_value = all_list[tags].find_element(By.XPATH, './dt').text.strip().upper()
                if tag_value == 'INTERIOR':
                    interior_area = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'BEDROOMS':
                    bedroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'FULL BATHS':
                    bathroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'PROPERTY TYPE':
                    property_type = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'YEAR BUILT':
                    year_build = all_list[tags].find_element(By.XPATH, './dd').text
                else:
                    pass
            except:
                pass
        try:
            address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[1]').text
        except:
            address = ''
        try:
            region_address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[1]').text
        except:
            region_address = ''
        try:
            postal_code = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            postal_code = np.nan
            
        try:
            furnished_status = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            furnished_status = np.nan
        try:
            facing = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[11]/div/h1/div[2]/div/span[4]').text
        except:
            facing = np.nan

        full_address = str(address)
        
        price_items = driver.find_elements(By.XPATH, '//span[@class="price__value  u-ignore-phonenumber"]')
        price_items_len = len(price_items)

        if price_items_len == 1 or price_items_len == 2:
            price = price_items[0].text
        elif price_items_len == 4:
            price = np.nan
        elif price_items_len ==5 or price_items_len ==6:
            price = price_items[0].text
        
        elif price_items_len ==7:
            price = price_items[0].text
            price_sqft = price_items[2].text
        else:
            pass
                
        amanity_list = []
        amenities = driver.find_elements(By.XPATH, '//span[@class="prop-description__amenities-list-item-text"]')

        garage = False
        for amenity in amenities:
            if 'garage' in amenity.text.lower():
                garage = True

        df1_row = [i,bhk_type,property_type,bedroom,bathroom,balcony,full_address,'Chicago',postal_code,'USA',landmark]
        df2_row = [i,interior_area,status,floor,transaction,year_build,price_sqft]
        df3_row = [i,furnished_status,facing,garage,ownership_type,booking_amt,price]

        df1_container.append(df1_row)
        df2_container.append(df2_row)
        df3_container.append(df3_row)

        print(df1_row)
        print(df2_row)
        print(df3_row)

        i+=1
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    next_page = driver.find_element(By.XPATH, '//a[@class="paging__item  paging__item--next  js-at-icon-paging  js-paging-next"]')
    next_page.click()

page 1
item no. 1
[1, nan, 'Single Family', '6', '5', nan, '1867 N Burling Street', 'Chicago', ',', 'USA', nan]
[1, '9,000 Sq Ft.', nan, nan, nan, '2024', '$1,111']
[1, nan, nan, False, nan, nan, '$10,000,000']
item no. 2
[2, nan, 'Single Family', '10', '8', nan, '2026 N Kenmore Avenue', 'Chicago', ',', 'USA', nan]
[2, '13,400 Sq Ft.', nan, nan, nan, '1885', '$746']
[2, nan, nan, False, nan, nan, '$9,995,000']
item no. 3
[3, nan, nan, nan, nan, nan, '500 W Superior Street, #2703', 'Chicago', ',', 'USA', nan]
[3, nan, nan, nan, nan, nan, '$890']
[3, nan, nan, False, nan, nan, '$8,900,000']
item no. 4
[4, nan, 'Single Family', '6', '6', nan, '52 E BELLEVUE Place', 'Chicago', ',', 'USA', nan]
[4, nan, nan, nan, nan, '1885', nan]
[4, nan, nan, False, nan, nan, '$7,900,000']
item no. 5
[5, nan, 'Single Family', '5', '6', nan, '1970 N Burling Street', 'Chicago', ',', 'USA', nan]
[5, '7,800 Sq Ft.', nan, nan, nan, nan, '$997']
[5, nan, nan, True, nan, nan, '$7,777,750']
item no. 6
[6, nan, 'S

In [42]:
print(len(df1_container), len(df2_container), len(df3_container))
for list in df1_container:
    df1.loc[len(df1)] = list

for list in df2_container:
    df2.loc[len(df2)] = list
    
for list in df3_container:
    df3.loc[len(df3)] = list

print(df1.shape)

240 240 240
(2338, 11)


###  For Algarve Location *****  288 done

In [43]:
# Opening website through a custom class using Seleniu,
driver_obj = open_website()
driver = driver_obj.OpenSite()

search_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[1]/div/div[1]/input')
search_bar.send_keys('Algarve')
time.sleep(2)

select_Algarve = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[2]/div/div/dl/dd')
select_Algarve.click()
time.sleep(3)

click_galary = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[2]/div[3]/div[1]/div/div/a[1]/i')
click_galary.click()

In [44]:
df1_container = []
df2_container = []
df3_container = []
i = 1
for page in range(0,24):
    time.sleep(3)
    print(f"page {page+1}")
    get_all_items = driver.find_elements(By.XPATH, '//div[@class="listing-item__image  listing-item__tabs-container  js-c-thumbnail-carousel"]/a[1]')
    
    for item in get_all_items:
        print(f"item no. {i}")
        try:
            item.send_keys(Keys.CONTROL + Keys.RETURN)
            driver.switch_to.window(driver.window_handles[1])
        except StaleElementReferenceException:
            print("Refreshing page because of StaleElementReferenceException")
            driver.refresh()
            time.sleep(1.5)
            continue

        bhk_type = property_type = bedroom = bathroom = balcony = full_address = postal_code = landmark = np.nan
        interior_area= status = floor = transaction = year_build = price_sqft = np.nan
        furnished_status = facing = garage = ownership_type = booking_amt = price= np.nan
        
        time.sleep(1.5)
        all_list = driver.find_elements(By.XPATH,'//dl[@class="listing-info__box-content"]')
        time.sleep(1.5)
        for tags in range(0,len(all_list)):
            try:
                tag_value = all_list[tags].find_element(By.XPATH, './dt').text.strip().upper()
                if tag_value == 'INTERIOR':
                    interior_area = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'BEDROOMS':
                    bedroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'FULL BATHS':
                    bathroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'PROPERTY TYPE':
                    property_type = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'YEAR BUILT':
                    year_build = all_list[tags].find_element(By.XPATH, './dd').text
                else:
                    pass
            except:
                pass
        try:
            address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[1]').text
        except:
            address = ''
        try:
            region_address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[1]').text
        except:
            region_address = ''
        try:
            postal_code = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            postal_code = np.nan
            
        try:
            furnished_status = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            furnished_status = np.nan
        try:
            facing = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[11]/div/h1/div[2]/div/span[4]').text
        except:
            facing = np.nan

        full_address = str(address)
        
        price_items = driver.find_elements(By.XPATH, '//span[@class="price__value  u-ignore-phonenumber"]')
        price_items_len = len(price_items)

        if price_items_len == 1 or price_items_len == 2:
            price = price_items[0].text
        elif price_items_len == 4:
            price = np.nan
        elif price_items_len ==5 or price_items_len ==6:
            price = price_items[0].text
        
        elif price_items_len ==7:
            price = price_items[0].text
            price_sqft = price_items[2].text
        else:
            pass
                
        amanity_list = []
        amenities = driver.find_elements(By.XPATH, '//span[@class="prop-description__amenities-list-item-text"]')

        garage = False
        for amenity in amenities:
            if 'garage' in amenity.text.lower():
                garage = True

        df1_row = [i,bhk_type,property_type,bedroom,bathroom,balcony,full_address,'Algarve',postal_code,'Portugal',landmark]
        df2_row = [i,interior_area,status,floor,transaction,year_build,price_sqft]
        df3_row = [i,furnished_status,facing,garage,ownership_type,booking_amt,price]

        df1_container.append(df1_row)
        df2_container.append(df2_row)
        df3_container.append(df3_row)

        print(df1_row)
        print(df2_row)
        print(df3_row)

        i+=1
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    next_page = driver.find_element(By.XPATH, '//a[@class="paging__item  paging__item--next  js-at-icon-paging  js-paging-next"]')
    next_page.click()

page 1
item no. 1
[1, nan, 'Other', nan, nan, nan, 'Tourist resort, for sale, in Almancil, Algarve', 'Algarve', 'Algarve', 'Portugal', nan]
[1, nan, nan, nan, nan, '2018', nan]
[1, nan, nan, False, nan, nan, nan]
item no. 2
[2, nan, 'Villa/Townhouse', nan, nan, nan, 'Property for sale in Ria Formosa, Algarve', 'Algarve', 'Algarve', 'Portugal', nan]
[2, nan, nan, nan, nan, '1987', nan]
[2, nan, nan, False, nan, nan, '$21,276,596']
item no. 3
[3, nan, 'Land/Lot', nan, nan, nan, 'Farm with ruins and orchard, Tavira, Algarve', 'Algarve', 'Algarve', 'Portugal', nan]
[3, '2,820.17 Sq Ft.', nan, nan, nan, nan, '$5,659']
[3, nan, nan, False, nan, nan, '$15,957,447']
item no. 4
[4, nan, 'Other', nan, nan, nan, 'Tourist resort, for sale, in Mexilhoeira Grande, Algarve', 'Algarve', 'Algarve', 'Portugal', nan]
[4, nan, nan, nan, nan, nan, nan]
[4, nan, nan, False, nan, nan, '$13,297,872']
item no. 5
[5, nan, 'Other', nan, nan, nan, 'Holiday Village for sale in Lagos, Algarve', 'Algarve', 'Algarve'

In [45]:
print(len(df1_container), len(df2_container), len(df3_container))
for list in df1_container:
    df1.loc[len(df1)] = list

for list in df2_container:
    df2.loc[len(df2)] = list
    
for list in df3_container:
    df3.loc[len(df3)] = list

print(df1.shape)

288 288 288
(2626, 11)


###  For Ontario Location *****  420 done

In [46]:
# Opening website through a custom class using Seleniu,
driver_obj = open_website()
driver = driver_obj.OpenSite()

search_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[1]/div/div[1]/input')
search_bar.send_keys('Ontario')
time.sleep(2)

select_Ontario = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[2]/div/div/dl[2]/dd')
select_Ontario.click()

time.sleep(3)

click_galary = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[2]/div[3]/div[1]/div/div/a[1]/i')
click_galary.click()

In [47]:
df1_container = []
df2_container = []
df3_container = []
i = 1
for page in range(0,35):
    time.sleep(3)
    print(f"page {page+1}")
    get_all_items = driver.find_elements(By.XPATH, '//div[@class="listing-item__image  listing-item__tabs-container  js-c-thumbnail-carousel"]/a[1]')
    
    for item in get_all_items:
        print(f"item no. {i}")
        try:
            item.send_keys(Keys.CONTROL + Keys.RETURN)
            driver.switch_to.window(driver.window_handles[1])
        except StaleElementReferenceException:
            print("Refreshing page because of StaleElementReferenceException")
            driver.refresh()
            time.sleep(1.5)
            continue

        bhk_type = property_type = bedroom = bathroom = balcony = full_address = postal_code = landmark = np.nan
        interior_area= status = floor = transaction = year_build = price_sqft = np.nan
        furnished_status = facing = garage = ownership_type = booking_amt = price= np.nan
        
        time.sleep(1.5)
        all_list = driver.find_elements(By.XPATH,'//dl[@class="listing-info__box-content"]')
        time.sleep(1.5)
        for tags in range(0,len(all_list)):
            try:
                tag_value = all_list[tags].find_element(By.XPATH, './dt').text.strip().upper()
                if tag_value == 'INTERIOR':
                    interior_area = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'BEDROOMS':
                    bedroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'FULL BATHS':
                    bathroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'PROPERTY TYPE':
                    property_type = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'YEAR BUILT':
                    year_build = all_list[tags].find_element(By.XPATH, './dd').text
                else:
                    pass
            except:
                pass

        try:
            address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[1]').text
        except:
            address = ''
        try:
            region_address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[1]').text
        except:
            region_address = ''
        try:
            postal_code = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            postal_code = np.nan
            
        try:
            furnished_status = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            furnished_status = np.nan
        try:
            facing = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[11]/div/h1/div[2]/div/span[4]').text
        except:
            facing = np.nan

        full_address = str(address)
        
        price_items = driver.find_elements(By.XPATH, '//span[@class="price__value  u-ignore-phonenumber"]')
        price_items_len = len(price_items)

        if price_items_len == 1 or price_items_len == 2:
            price = price_items[0].text
        elif price_items_len == 4:
            price = np.nan
        elif price_items_len ==5 or price_items_len ==6:
            price = price_items[0].text
        
        elif price_items_len ==7:
            price = price_items[0].text
            price_sqft = price_items[2].text
        else:
            pass
                
        amanity_list = []
        amenities = driver.find_elements(By.XPATH, '//span[@class="prop-description__amenities-list-item-text"]')

        garage = False
        for amenity in amenities:
            if 'garage' in amenity.text.lower():
                garage = True

        df1_row = [i,bhk_type,property_type,bedroom,bathroom,balcony,full_address,'Ontario',postal_code,'Canada',landmark]
        df2_row = [i,interior_area,status,floor,transaction,year_build,price_sqft]
        df3_row = [i,furnished_status,facing,garage,ownership_type,booking_amt,price]

        df1_container.append(df1_row)
        df2_container.append(df2_row)
        df3_container.append(df3_row)

        print(df1_row)
        print(df2_row)
        print(df3_row)

        i+=1
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    next_page = driver.find_element(By.XPATH, '//a[@class="paging__item  paging__item--next  js-at-icon-paging  js-paging-next"]')
    next_page.click()

page 1
item no. 1
[1, nan, nan, nan, nan, nan, '18 Chestnut Park Rd', 'Ontario', 'M4W1W6', 'Canada', nan]
[1, nan, nan, nan, nan, nan, nan]
[1, nan, nan, False, nan, nan, '$15,328,467']
item no. 2
[2, nan, 'Single Family', '12', nan, nan, '2615/2641 Ridge Road W', 'Ontario', 'L0L2L0', 'Canada', nan]
[2, nan, nan, nan, nan, nan, nan]
[2, nan, nan, False, nan, nan, '$13,503,650']
item no. 3
[3, nan, 'Single Family', '7', nan, nan, '84 Old Forest Hill Rd', 'Ontario', 'M5P2R5', 'Canada', nan]
[3, nan, nan, nan, nan, nan, nan]
[3, nan, nan, False, nan, nan, '$13,138,686']
item no. 4
[4, nan, 'Single Family', '5', nan, nan, '82 Chestnut Park Rd', 'Ontario', 'M4W2R3', 'Canada', nan]
[4, nan, nan, nan, nan, nan, nan]
[4, nan, nan, False, nan, nan, '$12,043,796']
item no. 5
[5, nan, 'Single Family', '7', nan, nan, '67 Hillholm Rd', 'Ontario', 'M5P1M4', 'Canada', nan]
[5, nan, nan, nan, nan, nan, nan]
[5, nan, nan, False, nan, nan, '$11,313,869']
item no. 6
[6, nan, 'Single Family', '5', nan, na

In [48]:
print(len(df1_container), len(df2_container), len(df3_container))
for list in df1_container:
    df1.loc[len(df1)] = list

for list in df2_container:
    df2.loc[len(df2)] = list
    
for list in df3_container:
    df3.loc[len(df3)] = list

print(df1.shape)

420 420 420
(3046, 11)


###  For Lisboa Location *****  420 done

In [52]:
# Opening website through a custom class using Seleniu,
driver_obj = open_website()
driver = driver_obj.OpenSite()

search_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[1]/div/div[1]/input')
search_bar.send_keys('Lisboa')
time.sleep(2)

select_lisboa = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[2]/div/div/dl[2]/dd')
select_lisboa.click()

time.sleep(3)

click_galary = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[2]/div[3]/div[1]/div/div/a[1]/i')
click_galary.click()

In [53]:
df1_container = []
df2_container = []
df3_container = []
i = 1
for page in range(0,35):
    time.sleep(3)
    print(f"page {page+1}")
    get_all_items = driver.find_elements(By.XPATH, '//div[@class="listing-item__image  listing-item__tabs-container  js-c-thumbnail-carousel"]/a[1]')
    
    for item in get_all_items:
        print(f"item no. {i}")
        try:
            item.send_keys(Keys.CONTROL + Keys.RETURN)
            driver.switch_to.window(driver.window_handles[1])
        except StaleElementReferenceException:
            print("Refreshing page because of StaleElementReferenceException")
            driver.refresh()
            time.sleep(1.5)
            continue
            
        bhk_type = property_type = bedroom = bathroom = balcony = full_address = postal_code = landmark = np.nan
        interior_area= status = floor = transaction = year_build = price_sqft = np.nan
        furnished_status = facing = garage = ownership_type = booking_amt = price= np.nan
        
        time.sleep(1.5)
        all_list = driver.find_elements(By.XPATH,'//dl[@class="listing-info__box-content"]')
        time.sleep(1.5)
        for tags in range(0,len(all_list)):
            try:
                tag_value = all_list[tags].find_element(By.XPATH, './dt').text.strip().upper()
                if tag_value == 'INTERIOR':
                    interior_area = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'BEDROOMS':
                    bedroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'FULL BATHS':
                    bathroom = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'PROPERTY TYPE':
                    property_type = all_list[tags].find_element(By.XPATH, './dd').text
                elif tag_value == 'YEAR BUILT':
                    year_build = all_list[tags].find_element(By.XPATH, './dd').text
                else:
                    pass
            except:
                pass

        try:
            address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[1]').text
        except:
            address = ''
        try:
            region_address = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[1]').text
        except:
            region_address = ''
        try:
            postal_code = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            postal_code = np.nan
            
        try:
            furnished_status = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[1]/div/h1/div[2]/div/span[4]').text
        except:
            furnished_status = np.nan
        try:
            facing = driver.find_element(By.XPATH, '/html/body/div[2]/d[1]/div/div/div/div[2]/div[11]/div/h1/div[2]/div/span[4]').text
        except:
            facing = np.nan

        full_address = str(address)
        
        price_items = driver.find_elements(By.XPATH, '//span[@class="price__value  u-ignore-phonenumber"]')
        price_items_len = len(price_items)

        if price_items_len == 1 or price_items_len == 2:
            price = price_items[0].text
        elif price_items_len == 4:
            price = np.nan
        elif price_items_len ==5 or price_items_len ==6:
            price = price_items[0].text
        
        elif price_items_len ==7:
            price = price_items[0].text
            price_sqft = price_items[2].text
        else:
            pass
                
        amanity_list = []
        amenities = driver.find_elements(By.XPATH, '//span[@class="prop-description__amenities-list-item-text"]')

        garage = False
        for amenity in amenities:
            if 'garage' in amenity.text.lower():
                garage = True

        df1_row = [i,bhk_type,property_type,bedroom,bathroom,balcony,full_address,'Lisboa',postal_code,'Portugal',landmark]
        df2_row = [i,interior_area,status,floor,transaction,year_build,price_sqft]
        df3_row = [i,furnished_status,facing,garage,ownership_type,booking_amt,price]

        df1_container.append(df1_row)
        df2_container.append(df2_row)
        df3_container.append(df3_row)

        print(df1_row)
        print(df2_row)
        print(df3_row)

        i+=1
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    next_page = driver.find_element(By.XPATH, '//a[@class="paging__item  paging__item--next  js-at-icon-paging  js-paging-next"]')
    next_page.click()

page 1
item no. 1
[1, nan, 'Estate', '9', '10', nan, 'Quinta da Alegria Estate', 'Lisboa', nan, 'Portugal', nan]
[1, '12,486 Sq Ft.', nan, nan, nan, nan, nan]
[1, nan, nan, True, nan, nan, nan]
item no. 2
[2, nan, 'Ranch/Farm', '12', '5', nan, 'Magnificent equestrian property in Sintra', 'Lisboa', nan, 'Portugal', nan]
[2, '27,099.45 Sq Ft.', nan, nan, nan, nan, nan]
[2, nan, nan, False, nan, nan, '$25,531,915']
item no. 3
[3, nan, 'Villa/Townhouse', '5', '8', nan, '5 Bedroom House, Cascais', 'Lisboa', nan, 'Portugal', nan]
[3, '5,328.18 Sq Ft.', nan, nan, nan, nan, '$2,995']
[3, nan, nan, True, nan, nan, '$15,957,447']
item no. 4
[4, nan, 'Villa/Townhouse', '8', '10', nan, '8 Bedroom House, Cascais', 'Lisboa', nan, 'Portugal', nan]
[4, '5,934.19 Sq Ft.', nan, nan, nan, nan, '$2,689']
[4, nan, nan, True, nan, nan, '$15,957,447']
item no. 5
[5, nan, 'Ranch/Farm', '12', '11', nan, '12 Bedroom Farm, Sintra', 'Lisboa', nan, 'Portugal', nan]
[5, '64,379.48 Sq Ft.', nan, nan, nan, nan, nan]


In [54]:
print(len(df1_container), len(df2_container), len(df3_container))
for list in df1_container:
    df1.loc[len(df1)] = list

for list in df2_container:
    df2.loc[len(df2)] = list
    
for list in df3_container:
    df3.loc[len(df3)] = list

print(df1.shape)

420 420 420
(3466, 11)


## For the above scraped data I have made some of the assumption for imputing values and I have stated my REASONING below for each assumption I have made, the assumption are made so I can have better SQL queries output result also this assumption making is purely based on facts and research I have done on US, Canada and Portugal housing system. This is not a part of EDA but just a way to have more meaningful data before performing sql or EDA because not every feature could be scraped and all the below statements are well researched facts about these features.

### Since many of the features are not listed on the website, there are still some assumptions we can make like :
### 1. For booking amount : for a property can range from 3-5% of the actual price.
### 2. For status : if the build year is equal to or greater than 2024 it can be 'Under construction' or 'New construction', if it is less than equal to 2014 it is in 'Repaired' category else it can be listed as 'Constructed'
### 3. For furnished status : it can be 'Unfurnished or Semi Furnished' for 'Sole owner' as the owner might take some or all of its belonging with him, 'Semi Furnished or Furnished' for 'Co-op' ownership
### 4. For facing : we can give weights to each direction based on statistics and research like South and East with 30% chance and North and West with 20% chance and then can pick a random facing direction for property to fill the null values
### 5. For ownership : The 'Single Family or Land/lot' property type can have a 'Sole owner' whereas 'realEstate or Land/lot' can have 'Co-op' ownership as well.

### * These are only assumptions that I took into consideration to make data more diversed and equiped with information according to my understanding. *

### Imputing assumptious data to make data more reliable and informative

In [55]:
df1.to_csv('csv1_scrapped.csv', index=False)
df2.to_csv('csv2_scrapped.csv', index=False)
df3.to_csv('csv3_scrapped.csv', index=False)

In [56]:
df1_copy = df1.copy()
df2_copy = df2.copy()
df3_copy = df3.copy()

print(df1_copy.shape, df2_copy.shape, df3_copy.shape)
print("df1_copy columns : ", df1_copy.columns)
print("df2_copy columns : ", df2_copy.columns)
print("df3_copy columns : ", df3_copy.columns)

(3466, 11) (3466, 7) (3466, 7)
df1_copy columns :  Index(['Sno', 'BHK', 'Property Type', 'beds', 'bathroom', 'Balcony', 'Address',
       'City', 'State code', 'Country', 'Landmark'],
      dtype='object')
df2_copy columns :  Index(['Sno', 'carpetarea', 'status', 'floor', 'transaction type',
       'Year Built', 'price per square feet'],
      dtype='object')
df3_copy columns :  Index(['Sno', 'furnished status', 'facing', 'Car Parking', 'Type of ownership',
       'booking amount', 'buy total price'],
      dtype='object')


In [57]:
df1_copy['Sno'] = range(1, len(df1_copy) + 1)
print(df1_copy.head(5), df1_copy.tail(5))

   Sno  BHK   Property Type beds bathroom  Balcony  \
0    1  NaN      Ranch/Farm  NaN      NaN      NaN   
1    2  NaN      Ranch/Farm  NaN      NaN      NaN   
2    3  NaN             NaN  NaN      NaN      NaN   
3    4  NaN             NaN  NaN      NaN      NaN   
4    5  NaN  Co-op property  NaN      NaN      NaN   

                                             Address   City State code  \
0  Farm with views over the River Douro, Sebolido...  Porto   4575-177   
1  Quinta d'Arauta-Agrotourism with vineyard, for...  Porto   4620-100   
2  Property with gardens, swimming pool and river...  Porto   4430-498   
3  Luxury villa for sale, with garden, by the sea...  Porto   4150-356   
4  Sale: building as new, store and 6 apartments,...  Porto   4150-695   

    Country  Landmark  
0  Portugal       NaN  
1  Portugal       NaN  
2  Portugal       NaN  
3  Portugal       NaN  
4  Portugal       NaN          Sno  BHK        Property Type beds bathroom  Balcony  \
3461  3462  NaN  Reside

In [58]:
df2_copy['Sno'] = range(1, len(df2_copy) + 1)
df3_copy['Sno'] = range(1, len(df3_copy) + 1)

In [65]:
# getting sno of rows which are missing total buy price for houses
print(df3_copy['buy total price'].isnull().sum())

null_buy_total_price = df3_copy[df3_copy['buy total price'].isnull()]
sno_values = null_buy_total_price['Sno']
sno_list = sno_values.tolist()
print(sno_list)

118
[1, 2, 3, 4, 14, 16, 17, 20, 21, 22, 25, 37, 38, 44, 45, 46, 47, 52, 53, 54, 55, 56, 57, 61, 62, 63, 64, 65, 67, 70, 71, 72, 73, 78, 80, 81, 86, 87, 97, 102, 112, 168, 171, 192, 193, 197, 216, 217, 226, 244, 247, 255, 272, 287, 332, 362, 386, 404, 426, 454, 469, 471, 482, 484, 508, 515, 552, 561, 598, 616, 617, 625, 631, 638, 643, 653, 676, 681, 688, 695, 715, 733, 744, 826, 892, 944, 963, 996, 1058, 1147, 1172, 1174, 1196, 1260, 1612, 1618, 2085, 2339, 2349, 2350, 2353, 2355, 2421, 2437, 2467, 2500, 2562, 2610, 2696, 2793, 2952, 3047, 3099, 3149, 3150, 3276, 3288, 3439]


In [67]:
# Filtering rows which have null values in as buy total price for houses
df3_copy = df3_copy[~df3_copy['Sno'].isin(sno_list)]
df2_copy = df2_copy[~df2_copy['Sno'].isin(sno_list)]
df1_copy = df1_copy[~df1_copy['Sno'].isin(sno_list)]
print(df3_copy['Sno'][:5])

4    5
5    6
6    7
7    8
8    9
Name: Sno, dtype: int64


In [68]:
# Removing $ and ',' from price and converting into int type
df3_copy['buy total price'] = df3_copy['buy total price'].replace({'\$': '', ',': ''}, regex=True).astype(int)
df3_copy['buy total price'][:5]

4    4408602
5    4268817
6    4166667
7    4086022
8    3897849
Name: buy total price, dtype: int32

In [70]:
# Removing $ and ',' from price per sq ft and converting into int type
df2_copy['price per square feet'] = df2_copy['price per square feet'].fillna('0')

df2_copy['price per square feet'] = df2_copy['price per square feet'].replace({'\$': '', ',': ''}, regex=True).astype(int)

In [72]:
# Removing ',' and Sq Ft from carpetarea column and converting into Int type
df2_copy['carpetarea'] = df2_copy['carpetarea'].fillna('0')

df2_copy['carpetarea'] = df2_copy['carpetarea'].replace({'[\,\sSqFt\.]': ''}, regex=True).astype(int)

In [73]:
print(df1_copy.shape, df2_copy.shape, df3_copy.shape)

(3348, 11) (3348, 7) (3348, 7)


In [74]:
# For booking_amount column
df3_copy['booking amount'] = df3_copy['buy total price'].apply(lambda x: int(x * np.random.uniform(0.03, 0.05)))

In [77]:
# For status column
import random

df2_copy['Year Built'] = df2_copy['Year Built'].fillna(0)
df2_copy['Year Built'] = df2_copy['Year Built'].astype(int)

def determine_status(year):
    if year >= 2024:
        return random.choice(['Under construction', 'New construction'])
    elif year <= 2014:
        return 'Repaired'
    else:
        return 'Constructed'

df2_copy['status'] = df2_copy['Year Built'].apply(determine_status)

In [88]:
# For facing column

directions = ['South', 'East', 'North', 'West']
weights = [0.30, 0.30, 0.20, 0.20]

# Function to fill null values based on weights
def fill_facing_value(row):
    if pd.isnull(row['facing']):
        return np.random.choice(directions, p=weights)
    return row['facing']

# Apply the function to fill null values
df3_copy['facing'] = df3_copy.apply(fill_facing_value, axis=1)
print(df3_copy['facing'][:10])

4     South
5     South
6      West
7     South
8     South
9     North
10     West
11    North
12    South
14    South
Name: facing, dtype: object


In [91]:
# For Ownership column
unique_values = df1_copy['Property Type'].unique()
print(unique_values)

ownership_mapping = {
    'Co-op property': 'Co-operative (Co-op)',
    'Ranch/Farm': 'Sole Ownership',
    'Villa/Townhouse': 'Sole Ownership',
    'Residence/Apartment': 'Sole Ownership',
    'Vineyard': 'Sole Ownership',
    'Land/Lot': 'Sole Ownership',
    'Single Family': 'Sole Ownership',
    'Commercial / Office': 'Corporate Ownership',
    'multi-unit': 'Co-ownership',
    'Private Island': 'Sole Ownership',
    'Condominium': 'Condominium',
    'Other': 'Unknown',
    'Vacant land': 'Sole Ownership',
    'Fractional Ownership Property': 'Fractional Ownership',
    'Estate': 'Sole Ownership',
    'Land / Commercial': 'Sole Ownership',
    'Multi Family': 'Co-ownership',
    'Land / Lots': 'Sole Ownership',
    'Building': 'Corporate Ownership'
}

df3_copy['Type of ownership'] = df1_copy['Property Type'].map(ownership_mapping)

df3_copy['Type of ownership'] = df3_copy['Type of ownership'].fillna('Unknown')

['Co-op property' 'Ranch/Farm' nan 'Villa/Townhouse' 'Residence/Apartment'
 'Vineyard' 'Land/Lot' 'Single Family' 'Commercial / Office' 'multi-unit'
 'Private Island' 'Condominium' 'Other' 'Vacant land'
 'Fractional Ownership Property' 'Estate' 'Land / Commercial'
 'Multi Family' 'Land / Lots' 'Building']


In [92]:
# For furnished_status column

furnished_mapping = {
    'Sole Ownership': ['Unfurnished', 'Semi Furnished', 'Furnished'],
    'Co-operative (Co-op)': ['Semi Furnished', 'Furnished'],
    'Corporate Ownership': ['Unfurnished', 'Semi Furnished'],
    'Condominium': ['Semi Furnished', 'Furnished'],
    'Co-ownership': ['Unfurnished', 'Semi Furnished', 'Furnished'],
    'Fractional Ownership': ['Semi Furnished', 'Furnished'],
    'Unknown': ['Unfurnished']
}


def determine_furnished_status(ownership):
    return np.random.choice(furnished_mapping[ownership])

df3_copy['furnished status'] = df3_copy['Type of ownership'].apply(determine_furnished_status)

In [93]:
df1_copy.head(10)

,Sno,BHK,Property Type,beds,bathroom,Balcony,Address,City,State code,Country,Landmark
4,5,NaN,Co-op property,NaN,NaN,NaN,"Sale: building as new, store and 6 apartments,...",Porto,4150-695,Portugal,NaN
5,6,NaN,Ranch/Farm,2,NaN,NaN,"Property with garden, lake and pool, for sale,...",Porto,4470-624,Portugal,NaN
6,7,NaN,NaN,NaN,NaN,NaN,Luxury 4 bedroom villa with pool and sea views...,Porto,4150-503,Portugal,NaN
7,8,NaN,Villa/Townhouse,5,7,NaN,"Magnificent villa with classic architecture, f...",Porto,4785-698,Portugal,NaN
8,9,NaN,Villa/Townhouse,4,5,NaN,Luxury 4 bedroom villa with pool and sea views...,Porto,4150-503,Portugal,NaN
9,10,NaN,Villa/Townhouse,4,5,NaN,Luxury 4 bedroom villa with pool and sea views...,Porto,4150-503,Portugal,NaN
10,11,NaN,Villa/Townhouse,4,5,NaN,Luxury 4 bedroom villa with pool and sea views...,Porto,4150-503,Portugal,NaN
11,12,NaN,Villa/Townhouse,4,6,NaN,"Four bedroom villa with pool, for sale, in Vil...",Porto,4480-757,Portugal,NaN
12,13,NaN,Villa/Townhouse,5,6,NaN,"Luxury villa with pool and garden, for sale, i...",Porto,4150-023,Portugal,NaN
14,15,NaN,Villa/Townhouse,5,7,NaN,"Villa with garden, pool and ocean views, for s...",Porto,4150-628,Portugal,NaN


In [99]:
# Converting currency values into INR

df2_copy['price per square feet'] = df2_copy['price per square feet'] * 83.12
df2_copy['price per square feet'] = df2_copy['price per square feet'].round().astype(int)
df2_copy.head(10)

,Sno,carpetarea,status,floor,transaction type,Year Built,price per square feet
4,5,0,Repaired,NaN,NaN,1987,0
5,6,0,Repaired,NaN,NaN,1938,0
6,7,0,Repaired,NaN,NaN,0,77551
7,8,0,Repaired,NaN,NaN,2006,0
8,9,448859,Repaired,NaN,NaN,0,72231
9,10,439171,Repaired,NaN,NaN,0,71733
10,11,438095,Repaired,NaN,NaN,0,71899
11,12,0,Constructed,NaN,NaN,2021,0
12,13,0,Repaired,NaN,NaN,2003,0
14,15,0,Constructed,NaN,NaN,2022,0


In [100]:
# Converting currency values into INR

df3_copy['buy total price'] = df3_copy['buy total price'] * 83.12
df3_copy['booking amount'] = df3_copy['booking amount'] * 83.12

df3_copy['buy total price'] = df3_copy['buy total price'].round().astype(int)
df3_copy['booking amount'] = df3_copy['booking amount'].round().astype(int)

df3_copy.head(10)

,Sno,furnished status,facing,Car Parking,Type of ownership,booking amount,buy total price
4,5,Furnished,South,False,Co-operative (Co-op),16290107,366442998
5,6,Unfurnished,South,False,Sole Ownership,12223045,354824069
6,7,Unfurnished,West,False,Unknown,15758887,346333361
7,8,Furnished,South,False,Sole Ownership,16372978,339630149
8,9,Semi Furnished,South,False,Sole Ownership,12145744,323989209
9,10,Furnished,North,False,Sole Ownership,14962182,315051648
10,11,Furnished,West,False,Sole Ownership,11880425,315051648
11,12,Unfurnished,North,False,Sole Ownership,10448018,312817216
12,13,Furnished,South,False,Sole Ownership,11646359,303879572
14,15,Furnished,South,False,Sole Ownership,11428169,294048138


### Converting dataframes into csv files

In [101]:
df1_copy.to_csv('Table1.csv', index=False)

In [102]:
df2_copy.to_csv('Table2.csv', index=False)

In [103]:
df3_copy.to_csv('Table3.csv', index=False)